# TP7 - Réduction de dimensions

Peut-on déduire le genre d'un film à son poster ? Nous nous proposons d'explorer cela en exploitant la réduction de dimension. On s'appuie sur le dataset publié sur [Kaggle](https://www.kaggle.com/datasets/raman77768/movie-classifier) lui-même extrait de ce [dataset](https://www.cs.ccu.edu.tw/~wtchu/projects/MoviePoster/index.html) construit pour répondre à la problématique que nous nous sommes posé, initialement avec des réseaux de neurones.

Commençons par construire un dataset exploitable pour nous. Nous allons devoir :
1. Charger l'image et son identifiant
2. Modifier l'image de sorte qu'elle soit de taille uniforme
3. Applatir l'image pour former le dataset tabulaire final

L'identifiant nous servira plus tard. Nous choisissons ici la taille standard (50, 50, 3) puisque les images sont en couleurs, cela nous amènera à un dataset de taille $75\times75\times3=7500$. Commençons par construire une fonction qui nous permettra de charger les images.

In [ ]:
from PIL import Image


path = "Multi_Label_dataset/Images"


def get_image(id, path=path):
    image_path = path + "/" + id + ".jpg"
    image = Image.open(image_path)
    return image

Construisons maintenant le dataset.

In [ ]:
import numpy as np
import pandas as pd
import os

dataset = []
images = os.listdir(path)
for file in images:
    image_path = path + "/" + file
    image = Image.open(image_path).resize((50, 50))
    dataset.append(np.array(image).flatten())

df = pd.DataFrame(dataset)
df["Id"] = [image.replace(".jpg", "") for image in images]
df.head()

Nous allons travailler avec l'algorithme UMAP, qui s'appuie sur une descente de gradient. Pour simplifier cette étape, nous proposons de préparer la matrice en conséquence.

**Consigne** : Après avoir créer une matrice *X* qui correspond à *df* sans la colonne *Id*, standardiser les données à l'aide de la classe [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

**Consigne** : Calculer les coordonnées dans la nouvelle base de chaque observations avec l'algorithme [UMAP](https://umap-learn.readthedocs.io/en/latest/). On précisera que l'on souhaite obtenir uniquement deux dimensions finale.

L'objectif a présent est d'exploiter la nouvelle base de deux manières :
1. **Visualiser** l'ensemble des affiches sur un même graphique, et leur répartition selon les genre
2. **Identifier** les affiches les plus proches selon la nouvelle base


## Objectif 1: Visualiser
Pour réussir le premier objectif, exploitons le dataset qui spécifie les genres.

In [ ]:
df_categories = pd.read_csv("Multi_Label_dataset/train.csv")
df_categories.head()

Nous avons à nouveau la colonne *Id* présente pour pouvoir faire la jointure. Notons que nous n'avons pas un unique genre pour chaque films. Ajoutons à notre *embedding* les informations de cette base de données.

**Consigne** : Définir une fonction `augment_embedding` qui prend en paramètre un *embedding* *X*. Elle ajoutera à cet embedding les identifiants issus du dataset *df* pour ensuite faire la jointure avec le dataset *df_categories*. On utilisera la méthode [`merge`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) pour la jointure, et la méthode [`dropna`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) pour supprimer les affiches dont on ne connait pas le genre.

Nous avons maintenant la capacité de visualiser l'embedding et d'y projeter un genre.

**Consigne** : Définir la fonction `plot_embedding_genre` qui prend en paramètre :
* *embedding*: un dataframe qui correspond à l'embedding apprit
* *genre*: chaîne de caractère qui correspond à une des colonnes de genre
* *title*: chaîne de caractère valant *None* par défaut, et qui permet de personnaliser le titre, sinon n'en affiche pas
La fonction affichera le scatter plot de chacune des affiches selon l'embedding apprit, et afficher de deux couleurs différentes les observations associées au genre d'intérêt

**Consigne** : Appliquer la fonction `plot_embedding_genre` sur plusieurs genre pour visualiser la base de données.

Jusqu'ici nous avons travailler avec les valeurs par défaut de l'algorithme UMAP, et donc visualiser qu'un unique embedding possible.

**Consigne** : Définir la fonction `learn_embedding` qui prend en paramètre les paramètres à transmettre à l'algorithme UMAP. Elle renverra un embedding apprit avec les paramètres renseigné pour UMAP, et il sera augmenté avec la fonction `augment_embedding`.

**Consigne** : Visualiser, pour un même genre, différent paramètre de UMAP.

## Objectif 2: Identifier

On souhaite à présent identifier pour une affiche donnée les affiches les plus proches. Pour définir *l'affiche la plus proche* on décide de travailler avec un embedding apprit, et deux possibilités de distance.
* **Distance euclidienne** : la distance classique, celle qui correspond au monde physique. Pour deux vecteurs $u, v \in \mathbb{R}^n$, la distance euclidienne $d$ est :
$$d(u, v) = \sqrt{\sum_{i=1}^n (u_i - v_i)^2}$$
* **Distance cosinus** : mesure de similarité entre deux vecteurs se basant sur l'angle entre les deux vecteurs. Pour deux vecteurs $u, v \in \mathbb{R}^n$, la distance cosinus $d$ est :
$$d(u, v) = 1 - \frac{\langle u, v\rangle}{\|u\|\|v\|}$$

**Consigne** : Définir les fonctions suivante, qui prendront en paramètres un embedding de dimension 2 et une observation:
1. `euclidean_distance`: renvoie la distance euclidienne entre l'observation et l'ensemble de l'embedding
2. `cosinus_distance`: renvoie la distance cosinus entre l'observation et l'ensemble de l'embedding

**Consigne** : Définir la fonction `view_neighbors` qui prendra en paramètre 
* *embedding*: une matrice qui correspond à l'embedding d'intérêt
* *row_index*: un entier qui correspond à l'indice de l'affiche d'intérêt
* *n_neighbors*: le nombre de voisin que l'on souhaite visualiser, par défaut 2
* *distance*: fonction qui correspond à la distance que l'on souhaite utiliser
* *figsize*: un tuple qui permet de contrôler la taille du graphique généré

Elle renverra l'affiche initiale et les *n_neighbors* associés selon la distance choisie pour l'embedding précisé. On pourra utiliser la méthode [`nsmallest`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.nsmallest.html) et la fonction `get_image` pour afficher les images dans leurs format initial.

**Consigne** : Exploiter la fonction `view_neighbors` et juger de la qualité de l'embedding.